In [26]:
import stim
import pymatching
import numpy as np
import matplotlib.pyplot as plt

In [ ]:



def surface_code_memory_experiment(circuit, N):
    
    # print(circuit)
    sampler = circuit.compile_detector_sampler()

    shots = N

    detection_events, actual_observable_flips = sampler.sample(shots, separate_observables=True)
    error_model = circuit.detector_error_model()

    matching = pymatching.Matching.from_detector_error_model(error_model)

    predicted_observable_flips = matching.decode_batch(detection_events)


    num_log_errors = np.sum(actual_observable_flips != predicted_observable_flips)

    logical_error_rate = num_log_errors / shots
    print(f"Logical error rate: {logical_error_rate} ({num_log_errors}/{shots})")

    return logical_error_rate



In [ ]:
def surface_code_err_count_experiment(circuit, N):
    """Count the number of physical errors in the surface code circuit."""
    sampler = circuit.compile_detector_sampler()

    shots = N

    detection_events, actual_observable_flips = sampler.sample(shots, separate_observables=True)
    error_model = circuit.detector_error_model()

    matching = pymatching.Matching.from_detector_error_model(error_model)

    predicted_observable_flips = matching.decode_batch(detection_events)

    actual_logical_flips = actual_observable_flips[0]
    actual_physical_flips = actual_observable_flips[1:]

    predicted_logical_flips = predicted_observable_flips[0]
    predicted_physical_flips = predicted_observable_flips[1:]

    num_logs_errors = np.sum(actual_logical_flips != predicted_logical_flips)

    # num_phys_errors += np.sum(actual_physical_flips == predicted_physical_flips)

    log_error_rate = num_logs_errors / shots
    print(f"Logical error rate: {log_error_rate} ({num_logs_errors}/{shots})")
    # phys_error_rate = num_phys_errors / shots
    # print(f"Physical error rate: {phys_error_rate} ({num_phys_errors}/{shots})")

    return log_error_rate



In [ ]:

def surface_code_circuit_transform(rounds, distance, virtual = False, phys = True):
    """Remove virtual measurements from the surface code circuit template. (guarded by virtual)"""
    """Add observables for each physical qubit (guarded by phys)"""
    circuit = stim.Circuit.generated("surface_code:rotated_memory_x", rounds = rounds, distance = distance, 
                                        after_clifford_depolarization = 0.003, after_reset_flip_probability= 0.001,
                                        before_measure_flip_probability= 0.001,
                                        )
    new_circuit = stim.Circuit()
    # Need physical errors or virtual measurement
    
    end1, end2 = 0, 0
    for index, op in enumerate(circuit):
        if op.name == "MX" and circuit[index-1].name != "MX":
            end1 = index
        if op.name != "MX" and circuit[index-1].name == "MX":
            end2 = index
            break
    # 
    new_circuit = circuit[:end1 - 1] + circuit[end1: end2] ## Remove error before virtual measurement 
    virtual_detector = circuit[end2:-1]
    
    if virtual == True: 
        new_circuit = new_circuit + virtual_detector

    # gap = 2 * distance + 1
    # new_circuit.append("MX", [(1+ i * gap) for i in range(distance)]) # type: ignore
    HX = surface_matrix_gen(distance)
    new_circuit.append("OBSERVABLE_INCLUDE", [stim.target_rec(-i * distance) for i in range(1, distance + 1)], 0) # type: ignore
    # for i in range(distance ** 2):
    #     new_circuit.append("OBSERVABLE_INCLUDE", [stim.target_rec(-i - 1)], i + 1) # type: ignore
    # new_circuit.append("OBSERVABLE_INCLUDE", [stim.target_rec(-i) for i in range(1, distance + 1)], 0) # type: ignore
    return circuit, new_circuit

circ, new_circ = surface_code_circuit_transform(1, 3, True, True)
print(new_circ)
log_error_rate = surface_code_err_count_experiment(new_circ, 10000)
print(log_error_rate)


QUBIT_COORDS(1, 1) 1
QUBIT_COORDS(2, 0) 2
QUBIT_COORDS(3, 1) 3
QUBIT_COORDS(5, 1) 5
QUBIT_COORDS(1, 3) 8
QUBIT_COORDS(2, 2) 9
QUBIT_COORDS(3, 3) 10
QUBIT_COORDS(4, 2) 11
QUBIT_COORDS(5, 3) 12
QUBIT_COORDS(6, 2) 13
QUBIT_COORDS(0, 4) 14
QUBIT_COORDS(1, 5) 15
QUBIT_COORDS(2, 4) 16
QUBIT_COORDS(3, 5) 17
QUBIT_COORDS(4, 4) 18
QUBIT_COORDS(5, 5) 19
QUBIT_COORDS(4, 6) 25
RX 1 3 5 8 10 12 15 17 19
Z_ERROR(0.001) 1 3 5 8 10 12 15 17 19
R 2 9 11 13 14 16 18 25
X_ERROR(0.001) 2 9 11 13 14 16 18 25
TICK
H 2 11 16 25
DEPOLARIZE1(0.003) 2 11 16 25
TICK
CX 2 3 16 17 11 12 15 14 10 9 19 18
DEPOLARIZE2(0.003) 2 3 16 17 11 12 15 14 10 9 19 18
TICK
CX 2 1 16 15 11 10 8 14 3 9 12 18
DEPOLARIZE2(0.003) 2 1 16 15 11 10 8 14 3 9 12 18
TICK
CX 16 10 11 5 25 19 8 9 17 18 12 13
DEPOLARIZE2(0.003) 16 10 11 5 25 19 8 9 17 18 12 13
TICK
CX 16 8 11 3 25 17 1 9 10 18 5 13
DEPOLARIZE2(0.003) 16 8 11 3 25 17 1 9 10 18 5 13
TICK
H 2 11 16 25
DEPOLARIZE1(0.003) 2 11 16 25
TICK
X_ERROR(0.001) 2 9 11 13 14 16 18 25
MR 2 

In [ ]:
### generate parity check matrix
import sympy
from collections import defaultdict
from itertools import combinations
def surface_matrix_gen(n):
    H = np.zeros((n * n - 1, 2 * n * n), dtype = int)
    t = n // 2
    halfrow = (pow(n, 2) - 1) // 2
    z_cnt = 0
    x_cnt = 0
    for i in range(t):
        i2 = 2 * i
        for j in range(t):
            j2 = 2 * j
            topl_od_z = i2 * n + j2 
            topl_ev_z = (i2 + 1) * n + (j2 + 1)
            topl_od_x = (i2) * n + (j2 + 1)
            topl_ev_x = (i2 + 1) * n + (j2)
            for k in [topl_od_x, topl_ev_x]:
                H[x_cnt][k] = 1
                H[x_cnt][k + 1] = 1
                H[x_cnt][k + n] = 1
                H[x_cnt][k + n + 1] = 1
                x_cnt += 1
            for k in [topl_od_z, topl_ev_z]:
                k1 = k + n * n
                H[halfrow + z_cnt][k1] = 1
                H[halfrow + z_cnt][k1 + 1] = 1
                H[halfrow + z_cnt][k1 + n] = 1
                H[halfrow + z_cnt][k1 + n + 1] = 1
                z_cnt += 1
        H[x_cnt][i2 * n] = 1
        H[x_cnt][(i2 + 1) * n] = 1
        x_cnt += 1
        temp = (i2 + 2) * n - 1 
        H[x_cnt][temp] = 1
        H[x_cnt][temp + n] = 1    
        x_cnt += 1
        H[halfrow + z_cnt][i2 + 1 + n * n] = 1
        H[halfrow + z_cnt][i2 + 2 + n * n] = 1
        z_cnt += 1
        temp = i2 + (2*n-1) * n
        H[halfrow + z_cnt][temp] = 1
        H[halfrow + z_cnt][temp + 1] = 1
        z_cnt += 1
    
    HX = H[:(n * n - 1)//2, :n*n]
    HZ = H[(n * n - 1)//2:, n*n:]
    return HX, HZ 

HX, HZ = surface_matrix_gen(3)

# def generate_prob(H, w_max, p):
def generate_prob(H, w_max):
    """Generate the probability of each error pattern."""
    p = sympy.Symbol('p')
    k, n = H.shape
    H = sympy.Matrix(H)
    dist = defaultdict(float)
    for w in range(w_max + 1):
        prob_weight = (p**w) * ((1 - p)**(n-w))
        # if prob_weight < 1e-10:
        #     break
        err_loc_iter = combinations(range(n), w)
        
        if w == 0:
            # snd_vec = np.zeros(k)
            snd_vec = sympy.zeros(k, 1)
            snd_tuple = tuple(snd_vec)
            dist[snd_tuple] += prob_weight
            continue

        for err_loc in err_loc_iter:
            cols = [H[:, i] for i in err_loc]
            snd_vec = sympy.zeros(k ,1)
            for col in cols: 
                snd_vec += col
            snd_vec_mod2 = [elem % 2 for elem in snd_vec]

            # snd_vec = np.sum(H[:, err_loc], axis = 1)% 2
        
            # snd_tuple = tuple(snd_vec)
            snd_tuple = tuple(snd_vec_mod2)
            dist[snd_tuple] += prob_weight
    
    return dist, p
# print(HX)
dist, p = generate_prob(HX, 1)
print(dist[(0,0,0,0)])


(1 - p)**9


In [17]:
import sympy
print(sympy.Symbol('p'))

p


In [ ]:

lerv = []
lert = []
for i in range(1, 5):
    circuit, new_circuit = surface_code_circuit_transform(1, 2 * i + 1)
    print(f"Running experiment with distance {2 * i + 1} and N = 10^7")
    lerv.append(surface_code_memory_experiment(circuit, 1000000))
    lert.append(surface_code_memory_experiment(new_circuit, 1000000))
    print("\n")  # Add a newline for better readability between experiments


#logarithmic plot
plt.figure(figsize=(10, 6))
plt.title("Logical Error Rate vs Distance")
plt.xlabel("Distance")
plt.ylabel("Logical Error Rate")
plt.xticks(range(1, 5))
plt.grid(True)
plt.yscale('log')
plt.xscale('linear')
plt.xticks(range(1, 5), [str(i) for i in range(1, 5)])
plt.plot(range(1, 5), lerv, marker='o', label='With Virtual Measurements')
plt.plot(range(1, 5), lert, marker='o', label='Without Virtual Measurements')
plt.legend()
# plt.show()
plt.savefig("Output/Figures/vtmeas_round1.png")


In [27]:
H = [0,0,0,1,1,1,0,0,0]

print(np.nonzero(H))

(array([3, 4, 5]),)
